In [18]:
import json
import plotly.graph_objects as go
from scipy.integrate import odeint
import numpy as np
import pandas as pd
import seaborn as sns
import sys
sys.path.append('../src') 
from separate_fitting import power

In [7]:
a21 = 1
a31 = 0.27
a32 = 0.73

a41 = 0.18
a42 = 0.24
a43 = 0.58

a51 = 0.24
a52 = 0.23
a53 = 0.2
a54 = 0.33

W2 = 63000
W3 = 20000
W4 = 15000
W5 = 33000
Ws2 = 8000

In [8]:
def system(state, t, x, P980, c1, c2, c3, c4, k31, k41, k51):

    ns2, n1, n2, n3, n4, n5 = state

    #ns2
    ms2 = 1.23*P980*(37*(100-x)-ns2) - Ws2*ns2 - (c1*n1+c2*n2+c3*n3+c4*n4)*ns2  # ns1 = total_Yb - ns2

    # n1
    m1 = -c1*n1*ns2 + a21*W2*n2 + a31*W3*n3 + a41*W4*n4 + a51*W5*n5 - k41*n1*n4 - k31*n1*n3 - k51*n5*n1

    # n2
    m2 = c1*n1*ns2 - c2*n2*ns2 - a21*W2*n2 + a32*W3*n3 + a42*W4*n4 + a52*W5*n5 + k41*n1*n4 + 2*k31*n1*n3

    # n3
    m3 = c2*n2*ns2 - c3*n3*ns2 - (a31+a32)*W3*n3 + a43*W4*n4 + a53*W5*n5 + 2*k51*n1*n5 + k41*n1*n4 - k31*n1*n3

    # n4
    m4 = c3*n3*ns2 - c4*n4*ns2 - (a43+a42+a41)*W4*n4 + a54*W5*n5 - k41*n1*n4

    # n5
    m5 = c4*n4*ns2 - (a54+a53+a52+a51)*W5*n5 - k51*n1*n5


    return [ms2, m1, m2, m3, m4, m5]

In [9]:
x=10
P980 = 6*10**3

In [10]:
with open(f'./../synthetic_data/syn_{x}.json', 'r') as f:
    loaded_json_data = f.read()
loaded_data = json.loads(loaded_json_data)
params = loaded_data.get('param', [])
experimental_data_NIR = loaded_data.get('NIR', [])
experimental_data_blue = loaded_data.get('blue', [])

In [11]:
c1, c2, c3, c4, k31, k41, k51 = params
c1, c2, c3, c4, k31, k41, k51 = 10**c1, 10**c2, 10**c3, 10**c4, 10**k31, 10**k41, 10**k51

In [12]:
y = 37 * x
state0 = [0, y, 0, 0, 0, 0]
t = np.arange(0.0, 0.001, 0.000001)
state = odeint(system, state0, t, args=(x, P980, c1, c2, c3, c4, k31, k41, k51))

In [17]:
ss = [state[:, i][-1] for i in range(6)]
ns2, n1, n2, n3, n4, n5 = ss

In [30]:
ss

[625.9604449753849,
 132.51885095333108,
 237.10813531380353,
 0.3530331773077807,
 0.004042848314338817,
 0.01593770724318207]

In [29]:
# c1*n1*ns2
# c2*n2*ns2
# c3*n3*ns2
# c4*n4*ns2
# k31*n1*n3
# k41*n1*n4
# k51*n1*n5

print([n1*ns2, n2*ns2, n3*ns2, n4*ns2, n1*n3, n1*n4, n1*n5])
print(a21*W2*n2)

[82951.55891037383, 148420.31388831223, 220.9848047586524, 2.5306631298115105, 46.7835510052307, 0.5357536131947915, 2.11204665069707]
14937812.524769623


In [22]:
print(c1*n1*ns2, c2*n2*ns2, c3*n3*ns2, c4*n4*ns2, k31*n1*n3, k41*n1*n4, k51*n1*n5)

14939561.78358791 7342.165675509951 696.368867157074 528.0275640490805 10.921066802086193 281.26020335135286 2.083157512521155


In [24]:
print(c1, c2, c3, c4, k31, k41, k51)

180.09983151407158 0.04946873836309903 3.1512070158742826 208.65185801652277 0.23343817575679854 524.9805067559873 0.9863217329189294


In [27]:
a32*W3*n3

5154.284388693599

In [28]:
a43*W4*n4

35.172780334747706